In [ ]:
import os
from pydna.dseqrecord import Dseqrecord
from pydna.assembly2 import (
    golden_gate_assembly,
    gibson_assembly,
    ligation_assembly,
    gateway_assembly,
    homologous_recombination_integration,
    homologous_recombination_excision,
    cre_lox_integration,
    cre_lox_excision,
    pcr_assembly,
)

from Bio.Restriction import BsaI, EcoRI, SalI
from pydna.parsers import parse_snapgene
from pydna.opencloning_models import CloningStrategy
from pydna.primer import Primer
from Bio.Seq import reverse_complement


## Golden Gate assembly (BsaI)
This section assembles three inserts into a circular vector using Golden Gate with BsaI and exports the design as JSON.


In [ ]:
insert1 = Dseqrecord("GGTCTCAattaAAAAAttaaAGAGACC")
insert2 = Dseqrecord("GGTCTCAttaaCCCCCatatAGAGACC")
insert3 = Dseqrecord("GGTCTCAatatGGGGGccggAGAGACC")

vector = Dseqrecord("TTTTattaAGAGACCTTTTTGGTCTCAccggTTTT", circular=True)

assembly_output = golden_gate_assembly(
    [insert1, insert2, insert3, vector], [BsaI], circular_only=True
)

cs = CloningStrategy.from_dseqrecord(assembly_output[0])

with open("golden_gate.json", "w") as f:
    f.write(cs.model_dump_json())

CloningStrategy.model_validate(cs.model_dump())


CloningStrategy(sequences=[TextFileSequence(id=5, type='TextFileSequence', sequence_file_format='genbank', overhang_crick_3prime=0, overhang_watson_3prime=0, file_content='LOCUS       name                      39 bp    DNA     circular UNK 01-JAN-1980\nDEFINITION  description.\nACCESSION   id\nVERSION     id\nKEYWORDS    .\nSOURCE      .\n  ORGANISM  .\n            .\nFEATURES             Location/Qualifiers\nORIGIN\n        1 attaaaaaat taacccccat atgggggccg gtttttttt\n//'), TextFileSequence(id=3, type='TextFileSequence', sequence_file_format='genbank', overhang_crick_3prime=0, overhang_watson_3prime=0, file_content='LOCUS       name                      27 bp    DNA     linear   UNK 01-JAN-1980\nDEFINITION  description.\nACCESSION   id\nVERSION     id\nKEYWORDS    .\nSOURCE      .\n  ORGANISM  .\n            .\nFEATURES             Location/Qualifiers\nORIGIN\n        1 ggtctcaatt aaaaaattaa agagacc\n//'), TextFileSequence(id=4, type='TextFileSequence', sequence_file_format='genbank'

## Gibson assembly
This section assembles three linear fragments using Gibson assembly and serializes the resulting product to JSON.


In [ ]:
fragments = [
    Dseqrecord("TTTTacgatAAtgctccCCCC", circular=False),
    Dseqrecord("CCCCtcatGGGG", circular=False),
    Dseqrecord("GGGGatataTTTT", circular=False),
]

products = gibson_assembly(fragments, limit=4)

cs = CloningStrategy.from_dseqrecord(products[0])
with open("gibson.json", "w") as f:
    f.write(cs.model_dump_json())


## Restriction/ligation assembly (EcoRI/SalI)
This section cuts a template with EcoRI and SalI, ligates compatible fragments, and exports the result as JSON.


In [ ]:
a = Dseqrecord("aaGAATTCccGTCGACaa")

c, d, e = a.cut(EcoRI, SalI)

f = ligation_assembly([c, d, e])

cs = CloningStrategy.from_dseqrecord(f[0])

with open("restriction_ligation.json", "w") as f:
    f.write(cs.model_dump_json())


### Gateway assembly (BP and LR)
This section creates an Entry vector via BP reaction, then generates an Expression clone via LR reaction using test SnapGene files.


In [ ]:
test_files = "tests"

line = "pDONRtm221.dna	pcr_product-attP1_1-attP2_1.dna	entry-attP1_1-attP2_1.dna	pET-53-DESTtm.dna	expression-attP1_1-attP2_1.dna".split(
    "\t"
)
backbone, pcr_product, _, backbone_expression, _ = line
backbone = parse_snapgene(os.path.join("tests/gateway_manual_cloning/" + backbone))[0]
pcr_product = parse_snapgene(
    os.path.join("tests/gateway_manual_cloning/" + pcr_product)
)[0]
backbone_expression = parse_snapgene(
    os.path.join("tests/gateway_manual_cloning/" + backbone_expression)
)[0]

# Works with the right reaction
entry_vector = gateway_assembly(
    [backbone, pcr_product], "BP", multi_site_only=True, circular_only=True
)[0]

expression_clone, _ = gateway_assembly(
    [backbone_expression, entry_vector], "LR", multi_site_only=True, circular_only=True
)

cs = CloningStrategy.from_dseqrecord(expression_clone)
with open("gateway.json", "w") as f:
    f.write(cs.model_dump_json())


## Homologous recombination integration
This section performs homologous recombination integration of two inserts into a genome and exports the result as JSON.

In [ ]:
homology = "AAGTCCGTTCGTTTTACCTG"
genome = Dseqrecord(f"aaaaaa{homology}ccccc{homology}aaaaaa")
insert = Dseqrecord(f"{homology}gggg{homology}")
integration_product, *_ = homologous_recombination_integration(genome, [insert], 20)

excision_product, *_ = homologous_recombination_excision(integration_product, 20)

cs = CloningStrategy.from_dseqrecord(excision_product)
with open("homologous_recombination.json", "w") as f:
    f.write(cs.model_dump_json())


## Cre-lox integration
This section performs Cre-lox integration , then excision.

In [ ]:
from pydna.cre_lox import LOXP_SEQUENCE
a = Dseqrecord(f"cccccc{LOXP_SEQUENCE}aaaaa")
b = Dseqrecord(f"{LOXP_SEQUENCE}bbbbb", circular=True)
integration_product, *_ = cre_lox_integration(a, [b])
excision_product, *_ = cre_lox_excision(integration_product)

cs = CloningStrategy.from_dseqrecord(excision_product)
with open("cre_lox.json", "w") as f:
    f.write(cs.model_dump_json())



## PCR assembly
This section performs PCR assembly and exports the result as JSON.


In [ ]:
primer1 = Primer("ACGTACGT")
primer2 = Primer(reverse_complement("GCGCGCGC"))

seq = Dseqrecord("ccccACGTACGTAAAAAAGCGCGCGCcccc")

prod, *_ = pcr_assembly(seq, primer1, primer2, limit=8)

cs = CloningStrategy.from_dseqrecord(prod)

with open("pcr.json", "w") as f:
    f.write(cs.model_dump_json())